# ALGORITMO DOWNLOAD PCAP

Il download è suddiviso in 5 fasi e sono il frutto di un processo di reverse engineering del processo di download della pagina contenente i pcap files (<a href='http://dataset.tlm.unavarra.es/ransomware/'>unavarra.es</a>).

1. ristrutturare json estratto dal sito;
2. download pcap files da <a href='https://www.malware-traffic-analysis.net/index.html'>malware-traffic-analysis</a>;
3. invio POST dataset.tlm.unavarra.es/ransomware/php/descargar.php;
4. acquisire risultato POST;
5. download pcap files da <a href='http://dataset.tlm.unavarra.es/download/'>dataset.tlm.unavarra.es/download/</a>.
6. Decomprimo i file

In [74]:
import os
#modulo per ricostruzione json
import json
#moduli per download pcap file
import requests as req
from bs4 import BeautifulSoup
import datetime
#modulo per decompressione file zip(.zip) e gzip(.gz)
from zipfile import ZipFile
import shutil
import gzip as gz

In [75]:
DS_BASE_PCAP = "/Volumes/Archivio/dataset/raw/pcap"

#LOG string
LOG_FAIL = "\tDownload failed"
LOG_SUCCESS = "\tDownload successfully"
LOG_NOTEXIST = "\tFile non exist"

## RISTRUTTURAZIONE JSON

Il file 'ransom.json' è stato estratto nel momento in cui si carica la pagina il quale contiene un elemento per ciascun sample analizzando il traffico di rete del browser con gli **strumenti da sviluppatore**.
<br>
<img src="./imgMD/response_json_php.png" width="850"/>
<br>
Alla fine di questo processo si vuole ottenere sempre un **json ristrutturato** con la seguente struttura in modo da raggruppare i vari sample per classe per poter replicare la struttura all'interno del file system:

~~~
{
    "famiglia1":[
        {
        "SamplePcap": "path",
        "Hash": "file_hash",
        "Link": "URL",
        "Scenario": "NAT o Original"
        },
        {"..."}
    ],

    "faliglia2": ["..."],

    "..." : ["..."],

    "faligliaN": ["..."]
}
~~~

* **SamplePcap**, contiene il percorso dove trovare il pcap file relativo al sample;
* **Hash**, contiene l'hash del sample preso in esame;
* **Link**, contiene URL dal quale è stato scaricato il binario (lo useremo per velocizzare il processo di download nel caso in cui il binario è stato scaricato da malware-traffic-analysis.net);
* **Scenario**, indica lo scenario utilizzato per l'analisi e può prendere due possibili valori: **Original o NAT**. Nel primo si è catturato solo il traffico tra client e server, invece nel secondo è stato catturato anche il traffico tra l'utente e internet.
<br>

<p align="center"><img src="./imgMD/scenario.png" width="850" align=center></p>
<p align="center">http://dataset.tlm.unavarra.es/ransomware/scenario_moreInfo_ioOps.html</p>

In [77]:
def serialize_json(filename, data):
    with open(f"{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {filename}")

def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}

In [ ]:
samples = "ransom.json"
samples_dict = read_json(samples)
samples_restructured = {}

for i in range(len(samples_dict)):
    family = samples_dict[str(i)][0]['Family']

    if family not in samples_restructured.keys():
        samples_restructured[family] = []

    sample = {
        "SamplePcap" : samples_dict[str(i)][0]['SamplePcap'],
        "Hash" : samples_dict[str(i)][0]['Hash'],
        "Link" : samples_dict[str(i)][0]['Link'],
        "Scenario" : samples_dict[str(i)][0]['Scenario']
    }

    samples_restructured[family].append(sample)

serialize_json('ransom_restructured.json', samples_restructured)

## DOWNLOAD PCAP FILES

In questa fase posso trovarmi in 2 possibbili scenari che variano a seconda della sorgente del file binario. La prima possibilità è che sia stato scaricato da <a href="https://www.malware-traffic-analysis.net/index.html">malware-traffic-analysis.net</a> (da ora lo abbreviamo che MTA), la seconda è <a href="https://www.hybrid-analysis.com/">hybrid-analysis.com</a> (abbreviamo con HA). Per velocizzare il processo di download nel primo caso andiamo a scaricare direttamente i pcap da MTA, invece nel secondo andiamo a scaricare i pcap da <a href="http://dataset.tlm.unavarra.es/ransomware/">univarra.es</a> eseguendo i seguenti passaggi:

1. invio POST a *dataset.tlm.unavarra.es/ransomware/php/descargar.php* con i seguenti parametri:
   * *tokenDescarga=*;
   * *descargarTodo=0*;
   * il valore di *sample* varia in base allo scenario in cui mi trovo.
2. acquisire la response della POST al *passo 1* in formato. Un esempio di response:
    ~~~
    {"name":"..\/downloads\/39e735642167b365b9546bf9e7cd27d5eb532f75.pcap.gz"}
    ~~~
   <br>
   La sequesnza di caratteri ha lunhezza fissa (40 caratteri), quidi è sicurante il risultato di una hash. Tale stringa però, a parità di valori degli attributi ad ogni esecuzione il risultato della funzione di hash cambio e ciò vuol dire che realizza un hash aggiunge una stringa che cambia sempre (esegue salting, concatena un timestamp, ...).
3. download risorsa (GET) all'indirizzo *dataset.tlm.unavarra.es/download/?*, dove al posto della wildcard (?) ci va il file indicato dalla POST al *passo 2*.

### DOWNLOA PCAP FILES DA MALWARE-TRAFFIC-ANALYSIS.NET

Il link per il download della risorsa viene reperito attraverso uno scrapping della pagina web. Dall' analisi dell'html si è notato che il link alla risorsa è un item di una lista presente sia all'inizio e sia alla fine della pagina (per comodità prendiamo la lista iniziale) ed è sempre il primo elemento della lista applicando il seguente filtro: **ul>li** oppure possiamo usare lal classe *menu_link*.
<br>
Qui di seguito il pezzo di html del sito a cui facciamo riferimento

~~~
<ul>
    <li>
        ZIP archive of the pcap:&nbsp;
        <a class="menu_link" href="2016-12-11-pseudoDarkleech-Rig-V-sends-Cerber-ransomware.pcap.zip">2016-12-11-pseudoDarkleech-Rig-V-sends-Cerber-ransomware.pcap.zip</a>
        &nbsp; 642 kB (642,326 bytes)
    </li>
</ul>
~~~

Il link alla fine avrà la seguente struttura: *https://www.malware-traffic-analysis.net/yyyy/mm/dd/nomeRisorsa*
<br>
Come parser è stato scelto *lxml* per il semplice fatto che non da problemi nel caso in cui il documento non sia ben formato.

In [114]:
def mta_download(url, dest, log_file):
    if not dest[-1] == '/':
        dest += '/'

    if not os.path.exists(dest):
        os.makedirs(dest, exist_ok=True)

    src = req.get(url).text
    soup = BeautifulSoup(src, 'lxml')

    name = soup.find('a', class_='menu_link').text

    #download pcapFile
    pcap_url = url[0 : len(url) - 10] + name
    res = req.get(pcap_url)

    print(pcap_url)
    log_file.write(f'{datetime.datetime.now()} {pcap_url}\n')
    if res.status_code == 200:
        print(LOG_SUCCESS)
        log_file.write(f'{LOG_SUCCESS}\n')
        with open(f'{dest + name}', 'wb') as pcap:
            pcap.write(res.content)
    else:
        print(LOG_FAIL)
        log_file.write(f'{LOG_FAIL}\n')

In [115]:
#TEST
with open('download.log', 'a') as log:
    mta_download("https://www.malware-traffic-analysis.net/2016/12/11/index.html", 'raw/pcap/test/', log)

ConnectionError: HTTPSConnectionPool(host='www.malware-traffic-analysis.net', port=443): Max retries exceeded with url: /2016/12/11/index.html (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x127909250>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

Nel caso in cui i percorso di destinazione non esiste, la funzione se lo crea automaticamente.

### DOWNLOA PCAP FILES DA UNAVARRA.ES

Come accennato in precedenza il download è suddiviso in 3 fasi. Nella POST va a restituire un stringa di 40 caratteri che sarà passato come parametro per la POST attraverso l'attributo *sample*.
<br>
*sample* conterrà il percorso del sample e ne esistono di due tipi in base allo scenario utilizzato per acquisire il traffico di rete:
1. *Scenario* = Original -> trazasOriginal/5GBdirectory/samplesPcap_compress/*SamplePcap*;
2. *Scenario* = NAT -> trazasNAT/samplesPcap_compress/*SamplePcap*.

In [73]:
#costruisce valore per attributo sample per la post
def build_sample_attr(sample_name, scenario):
    if scenario == "original":
        return f"trazasOriginal/5GBdirectory/{sample_name}"
    else:
        return f"trazasNAT/{sample_name}"

def ha_download(sample_name, scenario, dest, log_file):
    if not dest[-1] == '/':
        dest += '/'

    if not os.path.exists(dest):
        os.makedirs(dest, exist_ok=True)

    url1 = "http://dataset.tlm.unavarra.es/ransomware/php/descargar.php?" #usato per reperire il nome del file
    url2 = "http://dataset.tlm.unavarra.es/ransomware/downloads" #usato per scaricare il file
    sample_value = build_sample_attr(sample_name, scenario)

    payload = {'tokenDescarga' : '',
             'descargarTodo' : '0',
             'sample' : sample_value}
    header = {'Cookie' : 'PHPSESSID=eqbdqjkqrksd7u5l585mkk71t5',
              'Origin' : 'http://dataset.tlm.unavarra.es'} #necessario altrimenti mi restituiva un contenuto vuoto

    post_res = req.post(url1, data = payload, headers=header)

    if post_res.status_code == 200:
        post_res = post_res.json()['name'][12:]

        get_res = req.get(f'{url2 + post_res}')

        print(url2 + post_res)
        log_file.write(f'{datetime.datetime.now()} {url2 + post_res}\n')
        log_file.write(f'\tsample={sample_value}\n')

        if get_res.status_code == 200:
            print(LOG_SUCCESS)
            log_file.write(f'{LOG_SUCCESS}\n')
            with open(f'{dest + post_res}', 'wb') as pcap:
                pcap.write(get_res.content)
        else:
            print(LOG_FAIL)
            log_file.write(f'{LOG_FAIL}\n')
    else:
        print(LOG_NOTEXIST)
        log_file.write(f'{LOG_FAIL}\n')

In [14]:
#TEST
with open('download.log', 'a') as log:
    ha_download('samplesPcap_compress/cerber_10082016.pcap.gz', 'original', 'raw/pcap/test/', log)

http://dataset.tlm.unavarra.es/ransomware/downloads/444cdecb6d97ccc439c2be43a6a1eeea4e8fb3ed.pcap.gz
	Download successfully


Nel caso in cui i percorso di destinazione non esiste, la funzione se lo crea automaticamente.

## DECOMPRESSIONE ARCHIVI

Per la decompressione andiamo a utilizzare il modulo **ZipFile**.
Bisogna fare attenzione che i file con estensione *gz* non sono protetti da password, invece quelli con estensione *zip* lo sono (password: infected).
<br>
Per sicurezzo decidiamo di estrarre solo dile pcap nel caso in cui ci siano molteplici file con estensione diversa.

In [60]:
def unzip_pcap(path, type, pswd = None):
    splitted_path = path.split('/')

    dest_dir = "/".join(splitted_path[ : len(splitted_path) - 1])

    if type == 'zip':
        with ZipFile(path, 'r') as obj_zip:
            for file in obj_zip.namelist():
                if file.endswith('.pcap'): #necessaria perchè ci sono certi archivi che contengono alti file inerenti al ransonware oltre al pcap, che è ciò che mi interessa
                    obj_zip.extract(file, f'{dest_dir}/', pwd = bytes(pswd, 'utf-8'))
    elif type == 'gz':
        dest_name = splitted_path[-1]

        with gz.open(path, 'rb') as f_in:
            with open(f'{dest_dir}/{dest_name[: len(archive) - 3]}', 'wb') as f_out:
               shutil.copyfileobj(f_in, f_out)

In [ ]:
#TEST
for archive in os.listdir('raw/pcap/test'):
    if not archive == ".DS_Store":
        print(f"Decompressione {archive}...")
        if archive.endswith('zip'):
            unzip_pcap(f'raw/pcap/test/{archive}', pswd = 'infected', type = 'zip')
        elif archive.endswith('gz'):
            unzip_pcap(f'raw/pcap/test/{archive}', type = 'gz')

---
# DOWNLOAD TRAFFICO MALEVOLO

Costruite le 3 unità principale per il download (descritte sopra), andiamo a unire il tutto per ottenere l'algoritmo finale che è stato illustrato all'inizio.

In [ ]:
#Letture json
ransom_samples = read_json('ransom_restructured.json')

In [ ]:
######################################################## DA TESTARE #########################################################
#Download pcap files compressi
for family in ransom_samples.keys():
    for sample in ransom_samples[family]:
        path = f'{DS_BASE_PCAP}/malware/{family}/{sample["Hash"]}'

        if "malware-traffic-analysis.net" in sample["Link"]:
            with open('download.log', 'a') as log:
                mta_download(sample["Link"], path, log)
        elif "hybrid-analysis.com" in sample["Link"]:
            with open('download.log', 'a') as log:
                ha_download(sample["SamplePcap"], sample["Scenario"], path, log)

In [ ]:
######################################################## DA TESTARE #########################################################
#decompressione
for d in os.listdir(DS_BASE_PCAP):
    zip_path = f"{DS_BASE_PCAP}/{d}"

    for sub_d in os.listdir(zip_path):
        zip_path += f"/{sub_d}"

        for sub_sub_d in os.listdir(zip_path):
            zip_path += f"/{sub_sub_d}"

            for archive in os.listdir(zip_path):
                if not archive == ".DS_Store":
                    print(f"Decompressione {archive}...")

                    if archive.endswith('zip'):
                        unzip_pcap(f'{DS_BASE_PCAP}/{d}/{sub_d}/{sub_sub_d}/{archive}', pswd = 'infected', type = 'zip')
                    elif archive.endswith('gz'):
                        unzip_pcap(f'{DS_BASE_PCAP}/{d}/{sub_d}/{sub_sub_d}/{archive}', type = 'gz')

# DOWNLOAD TRAFFICO BENEVOLO

Il download del traffico benevole vengono scaricati da <a href="https://www.stratosphereips.org/datasets-overview">StratosphireLab</a>.
<br>
La struttura del link è la seguente: ...

<hr>

In [113]:
ran = read_json('ransom.json')

somma = 0
somma_no_mta = 0

for i in range(94):
    somma += float(ran[str(i)][0]["PCAPSize"])

    if "malware-traffic-analysis.net" not in ran[str(i)][0]["Link"]:
        somma_no_mta += float(ran[str(i)][0]["PCAPSize"])

print(f'MB totali: {somma}')
print(f'MB totali da pcap non presenti su mta: {somma_no_mta}')

Data read from path: ransom.json
MB totali: 276246.164
MB totali da pcap non presenti su mta: 166509.164
